In [49]:
from pulp import LpMaximize, LpMinimize, LpVariable, LpProblem, lpSum, LpStatus

### Problema de fluxo máximo e mínino

In [116]:
# primeiro será maximizado

prob = LpProblem('model', LpMaximize)

In [117]:
N = list(range(1, 7))

C = {
    1: {2: 16, 5: 13},
    2: {3: 12},
    3: {5: 9, 4: 20},
    5: {2: 4, 6: 14},
    6: {3: 7, 4: 4}}

D = {node: 0 for node in N} #definindo o lado direito das restrições

In [118]:
E = [(i,j) for i in N for j in N if i in C.keys() if j in C[i].keys()]

In [119]:
x = LpVariable.dicts('x', E,  lowBound = 0)

In [120]:
prob += x[(3, 4)] + x[(6, 4)]

prob += lpSum(x[1, 2]) - lpSum(x[2, 3]) == 0
prob += lpSum(x[1, 5]) + lpSum(x[3, 5]) - lpSum(x[5, 2]) == 0
prob += lpSum(x[2, 3]) + lpSum(x[6, 3]) - lpSum(x[3, 5]) - lpSum(x[3, 4]) == 0
prob += lpSum(x[5, 6]) - lpSum(x[6, 3]) - lpSum(x[6, 4]) == 0

#restrições de capacidade
for i in N:
  for j in N:
    if (i,j) in E:
      prob += x[i,j] <= C[i][j]

In [55]:
status = prob.solve()

# print("Valor da função objetivo:", prob.objective.value())

print(f'Status\n{LpStatus[status]}\n')

print('Valor ótimo das variáveis')
for v in prob.variables():
    print(v.name, '=', v.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cowvin/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bc9beaa0c2ec444da0f1b08efc4a0bd1-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/bc9beaa0c2ec444da0f1b08efc4a0bd1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 42 RHS
At line 56 BOUNDS
At line 57 ENDATA
Problem MODEL has 13 rows, 9 columns and 21 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-13) rows, 0 (-9) columns and 0 (-21) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 23
After Postsolve, objective 23, infeasibilities - dual 0.9999999 (1), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 23
Optimal - objective value 23
Optimal objective 23 - 0 iterations time 0.002, Presolve 0.00
Option for printingOpti

In [56]:
# parte da minimização

In [121]:
prob_min = LpProblem('minimiza_questao_1', LpMinimize)

In [122]:
prob_min += 16 * x[1, 2] + 13 * x[1, 5] + 12 * x[2, 3] + 4 * x[5, 2] + 9 * x[3, 5] + \
    14 * x[5, 6] + 7 * x[6, 3] + 4 * x[6, 4] + 20 * x[3, 4]

prob_min += x[1, 2] + x[1, 5] == 1
prob_min += x[1, 2] + x[5, 2] - x[2, 3] == 0
prob_min += x[2, 3] + x[6, 3] - x[3, 4] - x[3, 5] == 0
prob_min += x[1, 5] + x[3, 5] - x[5, 2] - x[5, 6] == 0
prob_min += x[5, 6] - x[6, 3] - x[6, 4] == 0
prob_min += x[3, 4] + x[6, 4] == 1

In [123]:
status_min = prob_min.solve()

# print("Valor da função objetivo:", prob.objective.value())

print(f'Status\n{LpStatus[status_min]}\n')

print('Valor ótimo das variáveis')
for v in prob_min.variables():
    print(v.name, '=', v.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cowvin/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5b79e62e34be4239a25740baa64b510c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/5b79e62e34be4239a25740baa64b510c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 39 RHS
At line 46 BOUNDS
At line 47 ENDATA
Problem MODEL has 6 rows, 9 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-4) rows, 4 (-5) columns and 8 (-10) elements
Optimal - objective value 31
After Postsolve, objective 31, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 31 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status
Optimal

Valor ótimo das variáveis
x_(1,_2) = 0.0
x

### Problema da designação

In [60]:
import pulp

# Definir os dados da matriz de custos
custos = [
    [10, 12, 15, 16],  # Tarefa 1
    [14, 12, 13, 18],  # Tarefa 2
    [10, 16, 19, 15],  # Tarefa 3
    [14, 12, 13, 15]   # Tarefa 4
]

# Definir o número de operários e tarefas
n_operarios = len(custos)
n_tarefas = len(custos[0])

# Criar o problema de minimização
problema = pulp.LpProblem("Problema_de_Designacao", pulp.LpMinimize)

# Criar variáveis de decisão: x[i][j] = 1 se o operário i for designado à tarefa j, caso contrário, 0
x = pulp.LpVariable.dicts("x", [(i, j) for i in range(n_operarios) for j in range(n_tarefas)], cat="Binary")

# Função objetivo: minimizar o tempo total de execução (somatório dos custos)
problema += pulp.lpSum([custos[i][j] * x[(i, j)] for i in range(n_operarios) for j in range(n_tarefas)])

# Restrições:
# 1. Cada operário deve realizar exatamente uma tarefa
for i in range(n_operarios):
    problema += pulp.lpSum([x[(i, j)] for j in range(n_tarefas)]) == 1

# 2. Cada tarefa deve ser realizada por exatamente um operário
for j in range(n_tarefas):
    problema += pulp.lpSum([x[(i, j)] for i in range(n_operarios)]) == 1

# Resolver o problema
problema.solve()

# Exibir a solução
print("Status:", pulp.LpStatus[problema.status])

# Exibir a designação de operários para as tarefas
for i in range(n_operarios):
    for j in range(n_tarefas):
        if pulp.value(x[(i, j)]) == 1:
            print(f"Operário {i+1} designado para Tarefa {j+1}, Custo: {custos[i][j]} horas.")

# Exibir o tempo total mínimo
print(f"Tempo total mínimo: {pulp.value(problema.objective)} horas.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cowvin/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/413089971a774c57840b9aab2779fdb8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/413089971a774c57840b9aab2779fdb8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 94 RHS
At line 103 BOUNDS
At line 120 ENDATA
Problem MODEL has 8 rows, 16 columns and 32 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 50 - 0.00 seconds
Cgl0004I processed model has 8 rows, 16 columns (16 integer (16 of which binary)) and 32 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 50
Cbc0038I Before mini branch and bound, 16 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not im

### Transbordo

In [112]:
prob_trans = pulp.LpProblem("Problema_de_Transbordo", pulp.LpMinimize)

# Definindo as variáveis de decisão
x_T1_T2 = pulp.LpVariable('x_T1_T2', lowBound=0, cat='Continuous')
x_D1_D2 = pulp.LpVariable('x_D1_D2', lowBound=0, cat='Continuous')
x_D2_D3 = pulp.LpVariable('x_D2_D3', lowBound=0, cat='Continuous')
x_P1_T1 = pulp.LpVariable('x_P1_T1', lowBound=0, cat='Continuous')
x_P1_T2 = pulp.LpVariable('x_P1_T2', lowBound=0, cat='Continuous')
x_P2_T1 = pulp.LpVariable('x_P2_T1', lowBound=0, cat='Continuous')
x_P2_T2 = pulp.LpVariable('x_P2_T2', lowBound=0, cat='Continuous')
x_T1_D1 = pulp.LpVariable('x_T1_D1', lowBound=0, cat='Continuous')
x_T1_D2 = pulp.LpVariable('x_T1_D2', lowBound=0, cat='Continuous')
x_T1_D3 = pulp.LpVariable('x_T1_D3', lowBound=0, cat='Continuous')
x_T2_D1 = pulp.LpVariable('x_T2_D1', lowBound=0, cat='Continuous')
x_T2_D2 = pulp.LpVariable('x_T2_D2', lowBound=0, cat='Continuous')
x_T2_D3 = pulp.LpVariable('x_T2_D3', lowBound=0, cat='Continuous')

In [113]:
prob_trans += (3 * x_P1_T1 + 4 * x_P1_T2 + 2 * x_P2_T1 + 5 * x_P2_T2 +
         8 * x_T1_D1 + 6 * x_T1_D2 + 4 * x_T2_D2 + 9 * x_T2_D3)

In [114]:
prob_trans += x_P1_T1 + x_P1_T2 == 1000, "Oferta_P1"
prob_trans += x_P2_T1 + x_P2_T2 == 1200, "Oferta_P2"

prob_trans += x_T1_D1 == 800, "Demanda_D1"
prob_trans += x_T1_D2 + x_T2_D2 == 900, "Demanda_D2"
prob_trans += x_T2_D3 == 500, "Demanda_D3"

prob_trans += x_P1_T1 + x_P2_T1 == x_T1_D1 + x_T1_D2 + x_T1_D3, "Equilibrio_T1"
prob_trans += x_P1_T2 + x_P2_T2 == x_T2_D1 + x_T2_D2 + x_T2_D3, "Equilibrio_T2"

In [115]:
prob_trans.solve()

# Status da solução
status = pulp.LpStatus[prob_trans.status]

# Valores ótimos das variáveis
solution = {v.name: v.varValue for v in prob_trans.variables()}

# Custo total mínimo
total_cost = pulp.value(prob_trans.objective)

print(f"Status: {status}")
print(f"Solução: {solution}")
print(f"Custo total mínimo: {total_cost}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cowvin/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2beed08d413844f4a7fb4b7c267bb6b0-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/2beed08d413844f4a7fb4b7c267bb6b0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 39 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 7 rows, 10 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-5) rows, 3 (-7) columns and 6 (-12) elements
0  Obj 24500 Primal inf 800 (1) Dual inf 3.999998 (2)
1  Obj 21700
Optimal - objective value 21700
After Postsolve, objective 21700, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 21700 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock sec